In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob

path = r'.\Data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# Data Preprocessing

At  the moment we have decided to filter the busiest airline, thereafter look at the top-3 most used airports to departure and arrival from/to. 

In [6]:
busiest_airline_id = frame['OP_CARRIER_AIRLINE_ID'].value_counts().head(1).index[0]
busiest_airline_id

19393

In [13]:
airports_filter = []
three_departure = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]['ORIGIN_AIRPORT_ID'].value_counts().head(3).index.to_list()
three_arrival = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]['DEST_AIRPORT_ID'].value_counts().head(3).index.to_list()
airports_filter.extend(x for x in three_departure if x not in airports_filter)
airports_filter.extend(x for x in three_arrival if x not in airports_filter)
airports_filter

[13232, 11292, 12889]

Filter df with busiest airline with top 3 departure, arrival airports

In [32]:
busiest_airline_df = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]
filtered_df = busiest_airline_df[(busiest_airline_df.DEST_AIRPORT_ID.isin(airports_filter)) | (busiest_airline_df.ORIGIN_AIRPORT_ID.isin(airports_filter))]
filtered_df

,YEAR,QUARTER,FL_DATE,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY
24037,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,1348.0,13.0,1454.0,9.0,0.0,NaN,NaN,NaN,NaN,NaN
24038,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,2038.0,28.0,2147.0,22.0,0.0,NaN,0.0,0.0,0.0,0.0
24039,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,543.0,-2.0,648.0,-12.0,0.0,NaN,NaN,NaN,NaN,NaN
24040,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,1058.0,23.0,1201.0,11.0,0.0,NaN,NaN,NaN,NaN,NaN
24041,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,1743.0,-2.0,1856.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355194,2022,3,7/13/2022 12:00:00 AM,19393,11638,11292,1458.0,3.0,1811.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
5355195,2022,3,7/13/2022 12:00:00 AM,19393,11638,12889,1544.0,-6.0,1652.0,-8.0,0.0,NaN,NaN,NaN,NaN,NaN
5355196,2022,3,7/13/2022 12:00:00 AM,19393,11638,12889,1226.0,71.0,1343.0,78.0,0.0,NaN,0.0,0.0,7.0,0.0
5355197,2022,3,7/13/2022 12:00:00 AM,19393,11638,12889,549.0,-6.0,654.0,-6.0,0.0,NaN,NaN,NaN,NaN,NaN


In [45]:
# Number of flights that had delay due to carrier 
filtered_df[(filtered_df['CARRIER_DELAY']>0)].shape

(58253, 16)

In [44]:
# Number of flights that had delay due to NAS 
filtered_df[filtered_df['NAS_DELAY']>0].shape

(32468, 16)

In [46]:
# Number of flights that had delay due to carrier and NAS 
filtered_df[(filtered_df['CARRIER_DELAY']>0)&(filtered_df['NAS_DELAY']>0)].shape


(18328, 16)

Fill in the delay columns that have nan with 0

In [53]:
filtered_df['CARRIER_DELAY'] = filtered_df['CARRIER_DELAY'].fillna(0)
filtered_df['NAS_DELAY'] = filtered_df['NAS_DELAY'].fillna(0)

c:\Users\tijnv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\tijnv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
filtered_dataframe.corr()

,YEAR,QUARTER,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY
YEAR,1.000000,0.015243,NaN,-0.013764,-0.013778,-0.012945,0.056194,-0.028252,0.055431,0.001823,0.014526,-0.018673,-0.047431,-0.005249
QUARTER,0.015243,1.000000,NaN,0.002323,0.001865,-0.009319,0.017767,-0.007966,0.016114,-0.099419,0.005050,0.042326,0.027894,0.006637
OP_CARRIER_AIRLINE_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ORIGIN_AIRPORT_ID,-0.013764,0.002323,NaN,1.000000,-0.085633,-0.020281,-0.014611,-0.002137,-0.011788,-0.001121,-0.027854,0.004316,0.002720,-0.001604
DEST_AIRPORT_ID,-0.013778,0.001865,NaN,-0.085633,1.000000,0.028850,0.014579,0.005486,0.026909,-0.001751,0.019766,-0.008330,-0.031788,-0.002598
DEP_TIME,-0.012945,-0.009319,NaN,-0.020281,0.028850,1.000000,0.256470,0.476769,0.252944,0.002942,-0.043303,-0.005472,-0.080331,-0.013665
DEP_DELAY,0.056194,0.017767,NaN,-0.014611,0.014579,0.256470,1.000000,0.006936,0.945683,0.019495,0.521946,0.174621,0.016194,0.034876
ARR_TIME,-0.028252,-0.007966,NaN,-0.002137,0.005486,0.476769,0.006936,1.000000,0.016735,NaN,-0.062346,0.002797,0.050032,0.003010
ARR_DELAY,0.055431,0.016114,NaN,-0.011788,0.026909,0.252944,0.945683,0.016735,1.000000,NaN,0.502512,0.195779,0.221335,0.033653
CANCELLED,0.001823,-0.099419,NaN,-0.001121,-0.001751,0.002942,0.019495,NaN,NaN,1.000000,NaN,NaN,NaN,NaN


In [14]:
filtered_dataframe.describe()

,YEAR,QUARTER,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY
count,399727.000000,399727.000000,399727.0,399727.000000,399727.000000,379465.000000,379465.000000,379360.000000,378480.000000,399727.000000,82872.000000,82872.000000,82872.000000,82872.000000
mean,2019.987291,2.276696,19393.0,12735.802117,12735.168385,1348.905512,12.458343,1413.414369,5.243720,0.050805,18.203157,1.233143,7.684091,0.089777
std,1.725624,0.447366,0.0,1308.610974,1308.861990,523.808719,31.063327,596.213074,33.516459,0.219599,31.738065,11.342274,18.825041,2.809640
min,2018.000000,2.000000,19393.0,10140.000000,10140.000000,1.000000,-52.000000,1.000000,-70.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,2.000000,19393.0,11292.000000,11292.000000,908.000000,-3.000000,1011.000000,-13.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020.000000,2.000000,19393.0,12889.000000,12889.000000,1336.000000,1.000000,1449.000000,-4.000000,0.000000,9.000000,0.000000,0.000000,0.000000
75%,2022.000000,3.000000,19393.0,13232.000000,13232.000000,1808.000000,15.000000,1909.000000,11.000000,0.000000,21.000000,0.000000,8.000000,0.000000
max,2022.000000,3.000000,19393.0,15624.000000,15624.000000,2400.000000,648.000000,2400.000000,697.000000,1.000000,648.000000,450.000000,418.000000,398.000000


In [56]:
#Select the columns that we will use and perform subgroup algorithm thingy